In [1]:
using Pkg: @pkg_str

pkg"activate .."
pkg"add BenchmarkTools"
pkg"status"

Activating environment at `~/JuliaEnvs/Arborist/Project.toml`
  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating registry at `~/.julia/registries/Invenia`
  Updating git-repo `git@gitlab.invenia.ca:invenia/PackageRegistry.git`
 Resolving package versions...
  Updating `~/JuliaEnvs/Arborist/Project.toml`
 [no changes]
  Updating `~/JuliaEnvs/Arborist/Manifest.toml`
 [no changes]
Project Arborist v0.1.0
    Status `~/JuliaEnvs/Arborist/Project.toml`
  [6e4b80f9] + BenchmarkTools v0.4.2
  [da1fd8a2] + CodeTracking v0.5.7
  [1914dd2f] + MacroTools v0.5.1
  [295af30f] + Revise v2.1.6
  [b77e0a4c] + InteractiveUtils 
    Status `~/JuliaEnvs/Arborist/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v0.4.2
  [da1fd8a2] + CodeTracking v0.5.7
  [1914dd2f] + MacroTools v0.5.1
  [295af30f] + Revise v2.1.6
  [b77e0a4c] + InteractiveUtils 


In [2]:
using Arborist
using Arborist: graft, get_ast
using Test
using BenchmarkTools
using MacroTools

┌ Info: Recompiling stale cache file /Users/oxinabox/.julia/compiled/v1.3/Arborist/OXQd1.ji for Arborist [a56570ba-ce5c-531d-8734-586fc97c4b8d]
└ @ Base loading.jl:1240


In [3]:
includet("../test/demo.jl")

In [18]:
@inline function fma_grafter(ast)
    MacroTools.prewalk(ast) do expr
        if (
            @capture(expr, x_ * y_ + z_) ||
            @capture(expr, z_ + x_ * y_)        
            )
            
            return :($x*fma($x, $y, $z)) 
        elseif isexpr(expr, :call)
            return Expr(:call,
                :graft, fma_grafter, expr.args...
            )
        end
        return expr
    end
end

Arborist.redeclare_graft()

graft (generic function with 3 methods)

In [21]:
bigmat = 100000rand((1000,1000))

@time graft(fma_grafter, fma_demo2, bigmat)

  0.000019 seconds (12 allocations: 288 bytes)


6.4000000000008e25

In [22]:
@time fma_demo2(bigmat)

  0.000017 seconds (5 allocations: 176 bytes)


8.000000000001e16